In [17]:
from Mesh import *
from sklearn.neighbors import KDTree
import numpy as np
import datetime

mesh = Mesh()
mesh.read_from_off('data/1.off')
# for i in range(10):
#     e = mesh.edges[i]
#     print(e.index,e, e.nh, e.ph)
mesh.set_face_normal()
mesh.set_vertex_normal()

v = []
n = []
for i in mesh.vertices:
    v.append(i.p)
    n.append(i.normal)
time = datetime.datetime.now()
# fpfh = FPFH(v, n)
print((datetime.datetime.now() - time))
time = datetime.datetime.now()
pfh = PFH(v, n)
print((datetime.datetime.now() - time))
print(sum(pfh))
print(sum(sum(pfh)))
time = datetime.datetime.now()
pfh = PFH2(v, n)
print((datetime.datetime.now() - time))
print(sum(pfh))
print(sum(sum(pfh)))

0:00:00.000024
0:00:07.983107
0:00:02.014473
0:00:10.040597
[0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 3.72260e+04 5.02000e+02 6.29940e+04 2.10640e+04
 2.16422e+05 1.55900e+04 8.00000e+02 4.23342e+05 1.64260e+04 8.43440e+04
 1.22000e+04 1.57800e+03 4.15278e+05 2.06000e+03 1.66040e+04 1.79500e+04
 1.47200e+03 5.45680e+04 6.80000e+02 1.13820e+04 1.51440e+04 8.44000e+02
 1.84040e+04 3.40000e+02 1.38700e+04 0.00000e+00 0.00000e+00 0.0

In [16]:
def FPFH(vertices, normals, r = 0.1):
    # default parameters
    sub = 11
    bins = 11 * 3
    l = len(vertices)
    
    vertices_xyz = []
    for v in vertices:
        vertices_xyz.append(v.data)
    # build kd tree
    tree = KDTree(vertices_xyz, leaf_size=2)  
    # find neighbours within distance of k for every vertex 
    ind = tree.query_radius(vertices_xyz, r, count_only = False, return_distance = False)
    # FPFH(p) = SPF(p) + 1/k * sigma( 1/w(k) * SPF(p(k)) )
    spf = np.zeros([l, bins])
    for i in range(l):
        pi = vertices[i]
        ni = normals[i]
        u = ni
        for j in ind[i]:
            if j <= i:
                continue
            #count u v w for pi and its neighbours (bj)
            pj = vertices[j]
            p_p = pj - pi
            v = Vector3D.cross_product(p_p, u)
            w = Vector3D.cross_product(u, v)
            nj = normals[j]
            
            #count alpha phi theta .., all in [-1, 1]
            alpha = Vector3D.dot_product(v, nj)
            phi = Vector3D.dot_product(u, p_p) * (1 / p_p.length)
            theta = math.atan2(Vector3D.dot_product(w, nj), Vector3D.dot_product(u, nj)) / math.pi
            
            #encode alpha phi theta into 33 bins, sub = 11 (default)
            index_a = int((alpha + 1) * sub / 2)
            if index_a > sub - 1:
                index_a = sub - 1
            index_b = int((phi + 1) * sub / 2)
            if index_b > sub - 1:
                index_b = sub - 1
            index_b += sub
            index_c = int((theta + 1) * sub / 2)
            if index_c > sub - 1:
                index_c = sub - 1
            index_c += sub * 2
            
            #count one time for each pair
            spf[i][index_a] += 1
            spf[i][index_b] += 1
            spf[i][index_c] += 1
            
            #for j, alpha_j = -alpha_i, phi_j = phi_i, theta_j = theta_i, so new index_a_j needed
            alpha = - alpha
            index_a = int((alpha + 1) * sub / 2)
            if index_a > sub - 1:
                index_a = sub - 1
            spf[j][index_a] += 1
            spf[j][index_b] += 1
            spf[j][index_c] += 1
    
    
    # FPFH(p) = SPF(p) + 1/k * sigma( 1/w(k) * SPF(p(k)) )
    fpfh = np.zeros([l, bins])        
    for i in range(l):
        fpfh[i] += spf[i][0]
        pi = vertices[i]
        k = len(ind[i]) - 1
        for j in ind[i]:
            if j == i:
                continue
            pj = vertices[j]
            w = (pi - pj).length
            fpfh[i] += spf[j] / w / k
    return fpfh

def PFH(vertices, normals, r = 0.1):
    # default parameters
    sub = 5
    bins = 5 ** 3
    l = len(vertices)
    
    vertices_xyz = []
    for v in vertices:
        vertices_xyz.append(v.data)
    # build kd tree
    tree = KDTree(vertices_xyz, leaf_size=2)  
    ind = tree.query_radius(vertices_xyz, r * 2, count_only = False, return_distance = False)
    time = datetime.datetime.now()
    #store the index of pi and pj in dict 
    apt = dict()
    for i in range(l):
        pi = vertices[i]
        ni = normals[i]
        u = ni
        for j in ind[i]:
            if i <= j:
                continue
            #count u v w for pi and its neighbours (bj)
            pj = vertices[j]
            p_p = pj - pi
            v = Vector3D.cross_product(p_p, u)
            w = Vector3D.cross_product(u, v)
            nj = normals[j]
            
            #count alpha phi theta ..
            alpha = Vector3D.dot_product(v, nj)
            phi = Vector3D.dot_product(u, p_p) * (1 / p_p.length)
            theta = math.atan2(Vector3D.dot_product(w, nj), Vector3D.dot_product(u, nj)) / math.pi
            
            #encode alpha phi theta into 125 bins, sub = 5 (default)
            index_a = int((alpha + 1) * sub / 2)
            if index_a > sub - 1:
                index_a = sub - 1
            index_b = int((phi + 1) * sub / 2)
            if index_b > sub - 1:
                index_b = sub - 1
            index_c = int((theta + 1) * sub / 2)
            if index_c > sub - 1:
                index_c = sub - 1
            index = index_a * sub * sub + index_b * sub + index_c
            code = "{}_{}".format(i,j)
            apt[code] = index
            
            #for j, alpha_j = -alpha_i, phi_j = phi_i, theta_j = theta_i, so new index_a_j needed
            alpha = -alpha
            index_a = int((alpha + 1) * sub / 2)
            if index_a > sub - 1:
                index_a = sub - 1
            index = index_a * sub * sub + index_b * sub + index_c
            #store the index of pi and pj in dict 
            code = "{}_{}".format(j,i)
            apt[code] = index
    
    print((datetime.datetime.now() - time))
    time = datetime.datetime.now()
    # find neighbours within distance of k for every vertex 
    ind = tree.query_radius(vertices_xyz, r, count_only = False, return_distance = False)
       
    pfh = np.zeros([l, bins])
    for ii in range(l):
        for i in ind[ii]:
            pi = vertices[i]
            ni = normals[i]
            u = ni
            for j in ind[ii]:
                if i == j:
                    continue
                code = "{}_{}".format(i,j)
                index = apt[code]
                pfh[ii][index] += 1
                
    print((datetime.datetime.now() - time))
    return pfh

def PFH2(vertices, normals, r = 0.1):
    # default parameters
    sub = 5
    bins = 5 ** 3
    l = len(vertices)
    
    vertices_xyz = []
    for v in vertices:
        vertices_xyz.append(v.data)
    # build kd tree
    tree = KDTree(vertices_xyz, leaf_size=2)  
            
    # find neighbours within distance of k for every vertex 
    ind = tree.query_radius(vertices_xyz, r, count_only = False, return_distance = False)
    pfh = np.zeros([l, bins])
    
    # use apt to store the pair that already counted
    apt =dict()
    for ii in range(l):
        for i in ind[ii]:
            pi = vertices[i]
            ni = normals[i]
            u = ni
            for j in ind[ii]:
                if i <= j:
                    continue
                code = "{}_{}".format(i,j)
                if code in apt:
                    index = apt[code]
                    pfh[ii][index] += 1
                    
                    code = "{}_{}".format(j,i)
                    index = apt[code]
                    pfh[ii][index] += 1
                else:
                    #count u v w for pi and its neighbours (bj)
                    pj = vertices[j]
                    p_p = pj - pi
                    v = Vector3D.cross_product(p_p, u)
                    w = Vector3D.cross_product(u, v)
                    nj = normals[j]

                    #count alpha phi theta .. all in [-1, -1]
                    alpha = Vector3D.dot_product(v, nj)
                    phi = Vector3D.dot_product(u, p_p) * (1 / p_p.length)
                    theta = math.atan2(Vector3D.dot_product(w, nj), Vector3D.dot_product(u, nj)) / math.pi

                    #encode alpha phi theta into 125 bins, sub = 5 (default)
                    index_a = int((alpha + 1) * sub / 2)
                    if index_a > sub - 1:
                        index_a = sub - 1
                    index_b = int((phi + 1) * sub / 2)
                    if index_b > sub - 1:
                        index_b = sub - 1
                    index_c = int((theta + 1) * sub / 2)
                    if index_c > sub - 1:
                        index_c = sub - 1
                    index = index_a * sub * sub + index_b * sub + index_c

                    #store the index of pi and pj in dict 
                    code = "{}_{}".format(i,j)
                    apt[code] = index
                    pfh[ii][index] += 1
                    
                    #for j, alpha_j = -alpha_i, phi_j = phi_i, theta_j = theta_i, so new index_a_j needed
                    alpha = -alpha
                    index_a = int((alpha + 1) * sub / 2)
                    if index_a > sub - 1:
                        index_a = sub - 1
                    index = index_a * sub * sub + index_b * sub + index_c
                    
                    code = "{}_{}".format(j,i)
                    apt[code] = index
                    pfh[ii][index] += 1
    return pfh

[[  0.           0.           0.         ...   1.77541037   4.79728244
   23.68028969]
 [  0.           0.           0.         ...   0.78352181   3.08330857
   45.68090037]
 [  0.           0.           0.         ...   4.68134296   5.68681896
  158.39489783]
 ...
 [  0.           0.           0.         ...   0.69473471   3.35286673
  181.24916565]
 [  0.           0.           0.         ...   7.25959207   6.72485349
  144.51388587]
 [  0.           0.           0.         ...   6.97613771   6.54765951
  165.32343145]]
